<a href="https://colab.research.google.com/github/stixmal/praktikum_project_ds/blob/main/%D0%97%D0%B0%D1%89%D0%B8%D1%82%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D1%81%D1%82%D1%80%D0%B0%D1%85%D0%BE%D0%B2%D0%BE%D0%B9%20%D0%BA%D0%BE%D0%BC%D0%BF%D0%B0%D0%BD%D0%B8%D0%B8/data_protection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Защита данных страховой компании

Защитим данные клиентов страховой компании «Хоть потоп». Разработали метод преобразования данных, усложняющий восстановление персональной информации. Обосновали корректность его работы. При преобразовании данных качество моделей машинного обучения не ухудшилось.  

Источник данных: `/datasets/insurance.csv`.

## Оглавление

### [1. Подготовка данных](#1) <a id='10'></a>   



### [2. Умножение матриц](#2) <a id='20'></a>



### [3. Алгоритм преобразования](#3) <a id='30'></a>



### [4. Проверка алгоритма](#4) <a id='40'></a>

***

## 1. Подготовка данных <a id='1'></a>   [^](#10) 

In [ ]:
# импорт библиотеки pandas и numpy
import pandas as pd 
import numpy as np  

# импорт модуля display
from IPython.display import display  

# отключение предупреждений
import warnings
warnings.filterwarnings('ignore') 

# импорт модуля разделения на выборки
from sklearn.model_selection import train_test_split

# импорт модели линейной регрессии
from sklearn.linear_model import LinearRegression

# импорт метрик
from sklearn.metrics import r2_score

In [ ]:
# чтение файла с данными
df = pd.read_csv('/datasets/insurance.csv')

# визуальный осмотр  
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


Признаки: 

* **Пол, Возраст, Зарплата, Члены семьи**  

Целевой признак:  

* **Страховые выплаты** - количество страховых выплат клиенту за последние 5 лет.

In [ ]:
# информация о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [ ]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Видим, что пропуски в данных отсутствуют, везде тип числовой. Соотношения минимума и максимума распределения в данных, а также небольшое различие среднего и медианы говорит о отсутствии неадекватных выбросов. 

Поделили выборку на трейн и тест.

In [ ]:
# переменные с признаками и ответами 
features = df.drop(['Страховые выплаты'], axis=1)
target = df['Страховые выплаты']

In [ ]:
# раздел на 75 % обучающей и 25 % тестовой выборки
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

Проконтролировали размер выборок.

In [ ]:
features_train.shape, target_train.shape

((3750, 4), (3750,))

In [ ]:
features_test.shape, target_test.shape

((1250, 4), (1250,))

In [ ]:
df.shape

(5000, 5)

## 2. Умножение матриц <a id='2'></a>  [^](#10) 

Ответим на вопрос -  
**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**  

- изменится 
- не изменится

Попробуем привести алгебраическое доказательство.

Предсказания линейной регрессии имеют вид: $$a = X·w = X·(X^TX)^{-1}·X^T·y $$  

где 
- a - предсказания; 
- X - матрица признаков;
- y - целевой признак.

Домножим матрицу признаков на обратимую квадратную случайную матрицу Z. Тогда формула расчета предсказаний линейной регрессии примет вид: $$a = X·Z·w = X·Z·((XZ)^T·(XZ))^{-1}·(XZ)^T·y$$  

По свойству обратной матрицы от произведения двух матриц получаем: 
$$a = X·Z·(XZ)^{-1}·((XZ)^T)^{-1}·(XZ)^T·y = X·Z·Z^{-1}·X^{-1}·E·y = X·X^{-1}·y$$

Тогда домножение обратимой квадратной матрицы на матрицу признаков не влияет на качество предсказаний.

## 3. Алгоритм преобразования <a id='3'></a>  [^](#10) 

Домножим нашу матрицу признаков на свою квадратную.
$$Xnew = (X·X^T)·X$$

In [ ]:
# преобразование признаков 
features_transform = np.array(features).dot(np.array(features).T).dot(np.array(features))
target_transform = target

In [ ]:
# раздел на 75 % обучающей и 25 % тестовой выборки
features_train_transform, features_test_transform, target_train_transform, target_test_transform = train_test_split(
    features_transform, target_transform, test_size=0.25, random_state=12345)

In [ ]:
# контроль выборок
features_train_transform.shape, target_train_transform.shape

((3750, 4), (3750,))

In [ ]:
features_test_transform.shape, target_test_transform.shape

((1250, 4), (1250,))

Создали план для преобразования и проверки алгоритма.

Алгоритм преобразования выполнили следующим образом: 

- Создаём рандомную квадратную матрицу, размерно позволяющую делать перемножение с матрицей признаков;
- Проверяем эту случайную матрицу на обратимость;
- Создаём новую матрицу данных;
- Проверяем качество предсказаний на исходных данных и преобразованных.

## 4 Проверка алгоритма <a id='4'></a>  [^](#10) 

 * ### Создание рандомной матрицы

In [ ]:
# создание рандомной матрицы с количеством строк и столбцов, равному количеству столбцов матрицы признаков
Z = np.random.rand(features.shape[1], features.shape[1])

 * ### Проверка случайной матрицы на обратимость

In [ ]:
# создание обратной матрицы
Z_inv = np.linalg.inv(Z)

In [ ]:
# проверка перемножения матриц
Z @ Z_inv

array([[ 1.00000000e+00,  6.43304747e-16,  4.47477904e-17,
         1.91145825e-16],
       [ 1.43448628e-16,  1.00000000e+00, -2.87232844e-17,
         4.96581050e-17],
       [ 3.38132243e-16, -1.11829388e-15,  1.00000000e+00,
         9.11663075e-17],
       [-4.98996410e-17, -5.65223752e-17,  5.39579338e-17,
         1.00000000e+00]])

Ответом является единичная матрица Е. Значит случайная матрица обратима.

 * ### Создание новой матрицы данных

In [ ]:
X = features @ Z

In [ ]:
# раздел на 75 % обучающей и 25 % тестовой выборки
X_train, X_test, target_train, target_test = train_test_split(
    X, target, test_size=0.25, random_state=12345)

In [ ]:
# контроль выборок
X_train.shape, target_train.shape

((3750, 4), (3750,))

In [ ]:
X_test.shape, target_test.shape

((1250, 4), (1250,))

 * ### Проверка качества предсказаний на исходных данных и преобразованных

Проверили преобразования матрицы признаков на метрике R2 после обучения и предсказания модели.

In [ ]:
# визуальный осмотр таблицы с признаками трейна
features_train

,Пол,Возраст,Зарплата,Члены семьи
3369,1,43.0,36200.0,1
1441,1,34.0,57600.0,0
571,0,32.0,41100.0,1
225,0,36.0,45100.0,1
2558,0,33.0,50600.0,2
...,...,...,...,...
3497,0,42.0,32100.0,0
3492,0,28.0,22700.0,4
2177,1,41.0,44700.0,1
3557,0,22.0,50100.0,4


In [ ]:
# предсказания модели и метрика R2 
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print(r2_score(target_test, predictions))

0.435227571270266


In [ ]:
# визуальный осмотр таблицы с преобразованными признаками трейна
X_train

,0,1,2,3
3369,20904.778180,25373.508530,26795.437276,6806.422907
1441,33256.613925,40367.197499,42627.616501,10814.869263
571,23730.978649,28804.864677,30418.737443,7719.604103
225,26040.697618,31608.382957,33379.276761,8471.268236
2558,29215.276854,35462.032452,37449.328356,9501.315253
...,...,...,...,...
3497,18537.134992,22499.843132,23760.210320,6036.290640
3492,13109.029120,15911.600655,16805.772904,4268.346342
2177,25811.191313,31329.278430,33084.586952,8399.269519
3557,28925.070231,35110.272755,37079.116858,9403.123272


In [ ]:
# предсказания модели и метрика R2
model = LinearRegression()
model.fit(X_train, target_train)
predictions = model.predict(X_test)
print(r2_score(target_test, predictions))

0.4352275712688335


## Вывод  

Мы провели преобразование признаков с помощью матричных операций. Качество линейной регрессии при этом не поменялось.